# Quick Start Tutorial of the basic functionality of MetaDrive

Welcome to MetaDrive!

MetaDrive v0.2.5 supports two running modes:

1. **With 3D rendering functionality**: MetaDrive can easily install and run on personal computer, but may need special treatments for 3D rendering in headless machines and cloud servers.

2. **Without 3D rendering functionality**: MetaDrive can easily be installed and run on any machine. In this Colab notebook, we mainly demonstrate MetaDrive in this mode and the renderer will be the **2D** **Pygame** renderer.

In this tutorial, we will navigate you through the installation and some basic functionality of MetaDrive!

## Installation

You can install MetaDrive easily.

In [ ]:
#@title Collect the MetaDrive

%pip install git+https://github.com/metadriverse/metadrive.git

In [ ]:
# @title Test Installation
from metadrive.engine.engine_utils import close_engine
close_engine()
# NOTE: usually you don't need the above lines. It is only for avoiding a potential bug when running on colab

from metadrive import MetaDriveEnv
from metadrive.examples import expert

env = MetaDriveEnv()
print("\nThe action space: {}".format(env.action_space))
print("\nThe observation space: {}\n".format(env.observation_space))
try:
    env.reset()
    for i in range(100):
        env.step(env.action_space.sample())
    print("Successfully run MetaDrive")
except:
    raise ValueError("Fail to run MetaDrive!")
finally:
    env.close()



## Basic Functionality

In [ ]:
# @title A minimalist example of using MetaDrive with the built-in PPO expert policy
from metadrive.engine.engine_utils import close_engine
close_engine()
# NOTE: usually you don't need the above lines. It is only for avoiding a potential bug when running on colab

import os
os.environ['SDL_VIDEODRIVER']='dummy'
# Note: this step is only for cheating Colab

from metadrive import MetaDriveEnv
from metadrive.examples import expert

# Set the envrionment config
config = {"start_seed": 1000, 
          "num_scenarios":1,
          "traffic_density":0.1,
          }

env = MetaDriveEnv(config)

print("Starting the environment ...\n")

ep_reward = 0.0
obs, info = env.reset()
for i in range(1000):
    obs, reward, terminated, truncated, info = env.step(expert(env.agent))
    ep_reward += reward
    env.render(mode="top_down", screen_record=True, screen_size=(500, 500))
    if terminated or truncated:
        print("Arriving Destination: {}".format(info["arrive_dest"]))
        print("\nEpisode reward: ", ep_reward)
        break

print("\nThe last returned information: {}".format(info))
env.top_down_renderer.generate_gif()
env.close()
print("\nMetaDrive successfully run!")

from IPython.display import Image
Image(open("demo.gif", 'rb').read())


In [ ]:
# @title PPO expert policy can also drive in the safety-critical SafeMetaDrive environment
from metadrive.engine.engine_utils import close_engine
close_engine()
# NOTE: usually you don't need the above lines. It is only for avoiding a potential bug when running on colab

import os
os.environ['SDL_VIDEODRIVER']='dummy'
# Note: this step is only for cheating Colab

from metadrive import SafeMetaDriveEnv
from metadrive.examples import expert

# Set the environment config
config = {"start_seed": 0, 
          "num_scenarios":1,
          "accident_prob":1.0,
          "traffic_density":0.25,
          }

env = SafeMetaDriveEnv(config)

print("Starting the environment ...\n")

ep_reward = 0.0
ep_cost = 0.0
obs, info = env.reset()
frames = []
for i in range(1000):
    obs, reward, terminated, truncated, info = env.step(expert(env.agent))
    ep_reward += reward
    ep_cost += info["cost"]
    frame = env.render(mode="top_down", window=False, screen_size=(500, 500))
    frames.append(frame)
    if terminated or truncated:
        print("Arriving Destination: {}".format(info["arrive_dest"]))
        print("\nEpisode reward: ", ep_reward)
        print("\nEpisode cost: ", ep_cost)
        break

env.close()
print("\nMetaDrive successfully run!")

# render image
print("\nGenerate gif...")
import pygame
import numpy as np
from PIL import Image

imgs = [frame for frame in frames]
imgs = [Image.fromarray(img) for img in imgs]
imgs[0].save("demo.gif", save_all=True, append_images=imgs[1:], duration=50, loop=0)
print("\nOpen gif...")
from IPython.display import Image
Image(open("demo.gif", 'rb').read())

In [ ]:
# @title Multi-agent Environment Visualization
from metadrive.engine.engine_utils import close_engine
close_engine()
# NOTE: usually you don't need the above lines. It is only for avoiding a potential bug when running on colab

import os
os.environ['SDL_VIDEODRIVER']='dummy'
# Note: this step is only for cheating Colab

from metadrive.envs.marl_envs import MultiAgentRoundaboutEnv, MultiAgentBottleneckEnv, MultiAgentIntersectionEnv, MultiAgentParkingLotEnv, MultiAgentTollgateEnv
from metadrive.examples import expert

env_classes = [MultiAgentRoundaboutEnv, 
        MultiAgentBottleneckEnv, 
        MultiAgentIntersectionEnv, 
        MultiAgentParkingLotEnv, 
        MultiAgentTollgateEnv]

frames = []
for env_class in env_classes:
    env = env_class()
    print("Starting the environment {}\n".format(env))
    env.reset()
    tm={"__all__":False}
    for i in range(100):
        if tm["__all__"]:
            frames.append(frame)
            continue
        action = env.action_space.sample()
        for a in action.values(): 
            a[-1] = 1.0
        o,r,tm,tc,i = env.step(action)
        frame = env.render(mode="top_down", 
                           scaling=4, # 4 pixels per meter
                           camera_position=env.current_map.get_center_point(), 
                           screen_size=(500, 500))
        frames.append(frame)
    env.close()

# render image
print("\nGenerate gif...")
import pygame
import numpy as np
from PIL import Image

imgs = [frame for frame in frames]
imgs = [Image.fromarray(img) for img in imgs]
imgs[0].save("demo.gif", save_all=True, append_images=imgs[1:], duration=50, loop=0)
print("\nOpen gif...")
from IPython.display import Image
Image(open("demo.gif", 'rb').read())

In [ ]:
# @title Real-world Scenario Environment Visualization
from metadrive.engine.engine_utils import close_engine
close_engine()
# NOTE: usually you don't need the above lines. It is only for avoiding a potential bug when running on colab

import argparse
import random
from metadrive.policy.replay_policy import ReplayEgoCarPolicy
from metadrive.constants import HELP_MESSAGE
from metadrive.engine.asset_loader import AssetLoader
from metadrive.envs import ScenarioEnv


class DemoEnv(ScenarioEnv):
    def reset(self, seed=None):
        if self.engine is not None:
            seeds = [i for i in range(self.config["num_scenarios"])]
            seeds.remove(self.current_seed)
            seed = random.choice(seeds)
        return super(DemoEnv, self).reset(seed=seed)


extra_args = dict(film_size=(1200, 1200))
asset_path = AssetLoader.asset_path

try:
    env = DemoEnv(
        {
            "manual_control": False,
            "reactive_traffic": False,
            "use_render": False,
            "agent_policy": ReplayEgoCarPolicy,
            "data_directory": AssetLoader.file_path(asset_path, "waymo", unix_style=False),
            "num_scenarios": 3
        }
    )
    o, _ = env.reset()
    frames = []
    for i in range(1, 100000):
        o, r, tm, tc, info = env.step([1.0, 0.])
        frame=env.render(
            mode="top_down",
            **extra_args
        )
        frames.append(frame)
        if tm or tc:
            env.reset()
            break
    
except Exception as e:
    raise e
finally:
    env.close()
        
# render image
print("\nGenerate gif...")
import pygame
import numpy as np
from PIL import Image

imgs = [frame for frame in frames]
imgs = [Image.fromarray(img) for img in imgs]
imgs[0].save("demo.gif", save_all=True, append_images=imgs[1:], duration=50, loop=0)
print("\nOpen gif...")
from IPython.display import Image
Image(open("demo.gif", 'rb').read())

## Map Generation

In [ ]:
# @title Draw the generated maps in top-down view
from metadrive.engine.engine_utils import close_engine
close_engine()
# NOTE: usually you don't need the above lines. It is only for avoiding a potential bug when running on colab


import random

import matplotlib.pyplot as plt
from metadrive import MetaDriveEnv
from metadrive.utils.draw_top_down_map import draw_top_down_map

env = MetaDriveEnv(config=dict(
    num_scenarios=100,
    map=7,
    start_seed=random.randint(0, 1000)
))

fig, axs = plt.subplots(4, 4, figsize=(10, 10), dpi=100)
for i in range(4):
    for j in range(4):
        env.reset()
        m = draw_top_down_map(env.current_map)
        ax = axs[i][j]
        ax.imshow(m, cmap="bone")
        ax.set_xticks([])
        ax.set_yticks([])
fig.suptitle("Bird's-eye view of generated maps")
plt.show()

env.close()


In [ ]:
# @title Draw the generated maps in top-down view with fixed block sequence
# @markdown You can also specify the road block sequence then randomize the block parameters.
# @markdown Please refer to [documentation](https://metadrive-simulator.readthedocs.io/en/latest/env_config.html#map-config) for the meaning of the map string.
from metadrive.engine.engine_utils import close_engine
close_engine()
# NOTE: usually you don't need the above lines. It is only for avoiding a potential bug when running on colab


import random

import matplotlib.pyplot as plt

from metadrive import MetaDriveEnv

env = MetaDriveEnv(config=dict(
    num_scenarios=100,
    map="CrTRXOS",
    start_seed=random.randint(0, 1000)
))

fig, axs = plt.subplots(4, 4, figsize=(10, 10), dpi=100)
for i in range(4):
    for j in range(4):
        env.reset()
        m = draw_top_down_map(env.current_map)
        ax = axs[i][j]
        ax = axs[i][j]
        ax.imshow(m, cmap="bone")
        ax.set_xticks([])
        ax.set_yticks([])
fig.suptitle("Bird's-eye view of generated maps")
plt.show()

env.close()


## 3D Renderer
For using the 3D renderer, just set `use_render=True` in the environment config. Or you can turn on the `image_observation`, which will make an offscreen renderer so we can access the rendered result by capturing frames.

In [ ]:
from metadrive.envs.metadrive_env import MetaDriveEnv
import os

cfg=dict(image_observation=True, 
         show_terrain=not os.getenv('TEST_IPYNB'),
         sensors={"main_camera": ()},
         vehicle_config=dict(image_source="main_camera"),
         window_size=(84, 60))

env=MetaDriveEnv(cfg)
try:
    env.reset()
    for _ in range(1 if os.getenv('TEST_DOC') else 5):
        env.step([0,0])
    env.capture("3d_renderer.png")

finally:
    env.close()

In [ ]:
from IPython.display import Image
Image(open("3d_renderer.png", "rb").read())